In [14]:
!pip install plotly
!pip install ipywidgets

In [2]:
import plotly
import anndata

import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import numpy as np

plotly.offline.init_notebook_mode(connected=True)


In [3]:
# adata = anndata.read_h5ad("data/big_table_with_embeddings_1000.h5ad")
adata = anndata.read_h5ad("data/big_table_with_embeddings_full.h5ad")


In [8]:
nan_vals = [np.nan, np.inf, -np.inf, "nan", "n/a", "na"]


In [9]:
# Add F:B ratio

f_cols = adata.var[adata.var["taxonomy_1"] == "p__Firmicutes"].index
b_cols = adata.var[adata.var["taxonomy_1"] == "p__Bacteroidetes"].index

f_vals = adata[:, f_cols].X.sum(axis=1)
b_vals = adata[:, b_cols].X.sum(axis=1) + 1e-6

fb_ratio = f_vals / b_vals
adata.obs["fb_ratio"] = np.clip(fb_ratio, 0, 10)


In [16]:
import plotly.express as px
from ipywidgets.embed import embed_minimal_html

obsm_matrices = list(adata.obsm.keys())
metadata_vars = list(adata.obs.keys())

# Create two blank figures, one for NaN values and one for regular data
# This has colorbar set to None so that it doesn't show up
pdisk = go.Scattergl(
    x=[],
    y=[],
    mode="lines",
    line=dict(color="black"),
    name="Poincare disk boundary",
)
scatter_nan = go.Scattergl(
    x=[],
    y=[],
    mode="markers",
    text=[],
    marker=dict(color="gray", colorbar=None, size=4),
    name="NaN values",
)
scatter = go.Scattergl(
    x=[],
    y=[],
    mode="markers",
    text=[],
    marker=dict(color=[], colorscale="Plasma", size=4, colorbar=dict(title="")),
    name="Valid values",
)

fig = go.FigureWidget(
    data=[scatter_nan, scatter, pdisk],
    layout=go.Layout(
        title=dict(text="Plotting Embeddings"),
        xaxis=dict(title="X"),
        yaxis=dict(title="Y"),
        hovermode="closest",
        # Move legend to bottom right
        legend=dict(x=1.1, y=0.5, bgcolor="rgba(0,0,0,0)"),
        autosize=False,
        width=1000,
        height=1000,
    ),
)

# Create widgets: select embedding and metadata, click plot
embed_dropdown = widgets.Dropdown(
    options=obsm_matrices, value=obsm_matrices[0], description="Embedding:"
)
metadata_dropdown = widgets.Dropdown(
    options=metadata_vars, value=metadata_vars[0], description="Metadata:"
)
plot_button = widgets.Button(description="Plot")

# Function to find nans in metadata
def find_nans(col):
    mask1 = col.isin(nan_vals)
    mask2 = col.isnull()
    return mask1 | mask2


# For Poincare disk
t = np.linspace(0, 2 * np.pi, 100)
circle_x = np.cos(t)
circle_y = np.sin(t)

# Update function: change embeddings, color, text
def plot_on_click(b):
    embedding = embed_dropdown.value
    x = adata.obsm[embedding][:, 0]
    y = adata.obsm[embedding][:, 1]
    if embedding == "h2":
        fig.data[2].x = circle_x
        fig.data[2].y = circle_y
    else:
        fig.data[2].x = []
        fig.data[2].y = []

    # Add color based on metadata
    metadata = metadata_dropdown.value
    col = adata.obs[metadata]
    if adata.obs[metadata].dtype.name == "category":
        col = col.str.lower()
        col = col.astype("category")
        color = col.cat.codes.copy()
    else:
        color = col.copy()

    nan_mask = find_nans(col)
    fig.data[0].x = x[nan_mask]
    fig.data[0].y = y[nan_mask]
    fig.data[1].marker.color = color[~nan_mask]
    fig.data[1].marker.colorbar.title = metadata
    fig.data[1].marker.colorbar.titleside = "top"
    fig.data[1].x = x[~nan_mask]
    fig.data[1].y = y[~nan_mask]

    # Update text to include metadata
    fig.data[0].text = [
        f"Index: {i}<br>{metadata}: {j}"
        for i, j in zip(adata.obs.index[nan_mask], col[nan_mask])
    ]
    fig.data[1].text = [
        f"Index: {i}<br>{metadata}: {j}"
        for i, j in zip(adata.obs.index[~nan_mask], col[~nan_mask])
    ]


plot_button.on_click(plot_on_click)

vbox = widgets.VBox([embed_dropdown, metadata_dropdown, plot_button])
vbox_all = widgets.VBox([vbox, fig])
display(vbox_all)

# Save plot
# embed_minimal_html("plot.html", views=[vbox_all], title="Mixture embeddings AGP")